**this is still being worked on**

https://imerit.net/blog/28-free-image-datasets-for-computer-vision-all-pbm/

https://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html

In [9]:
print("Installing and importing packages...")

# Uninstall and reinstall packages for a clean environment
!pip uninstall -q -y jhutils
!pip uninstall -q -y bs4
!pip install -q --disable-pip-version-check git+https://github.com/jdchart/jh-py-utils.git
!pip install -q --disable-pip-version-check beautifulsoup4

# Import packages
from jhutils.local_files import read_json, write_json, get_audio_info, get_video_info, get_image_info, collect_files
import jhutils.online_files
from jhutils.misc import print_progress_bar
import os
import shutil
import requests
from bs4 import BeautifulSoup

print("👍 Ready!")

Installing and importing packages...
👍 Ready!


In [10]:
SOURCE = "/Users/jacob/Documents/Repos/arvest-data-in-context/ml-notebooks/test-corpora/flowers"
OUT = {"images" : []}
PREFIX = "https://raw.githubusercontent.com/arvest-data-in-context/ml-notebooks/refs/heads/main/test-corpora/flowers/"
OURPATH = "/Users/jacob/Documents/Repos/arvest-data-in-context/ml-notebooks/_local/workflows/corpora/01-creating-an-image-corpus/random-flowers.json"

files_list = collect_files(SOURCE, ["jpg"])

for item in files_list:
    info = get_image_info(item)
    OUT["images"].append({
        "url" : f"{PREFIX}{os.path.basename(item)}",
        "data" : info
    })

write_json(OURPATH, OUT)

In [8]:
import os
import random
import shutil

def select_and_copy_files(input_folder, output_folder, num_files):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get list of files in input folder
    all_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]
    
    if num_files > len(all_files):
        raise ValueError("Requested more files than are available in the input folder.")

    # Randomly select files
    selected_files = random.sample(all_files, num_files)

    # Copy files to output folder
    for file_name in selected_files:
        src = os.path.join(input_folder, file_name)
        dst = os.path.join(output_folder, file_name)
        shutil.copy2(src, dst)

    print(f"Copied {len(selected_files)} files to '{output_folder}'.")

select_and_copy_files("/Users/jacob/Downloads/jpg", "/Users/jacob/Documents/Repos/arvest-data-in-context/ml-notebooks/test-corpora/flowers", 200)

Copied 200 files to '/Users/jacob/Documents/Repos/arvest-data-in-context/ml-notebooks/test-corpora/flowers'.


## Wikimedia commons

In [3]:
COLLECTION_URL = "https://commons.wikimedia.org/wiki/Category:JPG_photos"

TEMP_FOLDER = os.path.join(os.getcwd(), "_TEMP")
OUTPUT_PATH = os.path.join(os.getcwd(), "wikimedia-commons-small-jpg.json")

print(f"Processing {COLLECTION_URL}")

# Get urls
def process_page(url):
    response = requests.get(f"{url}")
    if response.status_code != 200:
        return None
    else:
        soup = BeautifulSoup(response.text, 'html.parser')
        element = soup.find(class_ = "gallery")
        if not element:
            return None
        else:
            images = element.find_all('img')
            img_srcs = [img['srcset'] for img in images if 'srcset' in img.attrs]
            if not img_srcs:
                return None
            return img_srcs
        
image_list = process_page(COLLECTION_URL)
parsed_images = []

for item in image_list:
    without_end = item.replace(' 1.5x','')

    new_prefix = without_end.replace('https://upload.wikimedia.org/wikipedia/commons/thumb/', 'https://upload.wikimedia.org/wikipedia/commons/')
    actual_path = new_prefix.split(".jpg/")[0] + ".jpg"

    parsed_images.append(without_end)

if os.path.isdir(TEMP_FOLDER) == False:
    os.makedirs(TEMP_FOLDER)

output_data = {"images" : []}

for i, image_url in enumerate(parsed_images):
    #print_progress_bar(i, len(parsed_images) - 1, f"Treating {os.path.basename(image_url)}")
    
    print(image_url)

    to_add = {"url" : image_url}
    dl_path = jhutils.online_files.download(image_url, dir = TEMP_FOLDER)
    
    if os.path.isfile(dl_path):
        img_info = get_image_info(dl_path)
        to_add["data"] = img_info

        output_data["images"].append(to_add)
        write_json(OUTPUT_PATH, output_data)

shutil.rmtree(TEMP_FOLDER)

Processing https://commons.wikimedia.org/wiki/Category:JPG_photos
https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/2023_08_Petrusstr_11_2.jpg/250px-2023_08_Petrusstr_11_2.jpg
Unexpected status code: 403
https://upload.wikimedia.org/wikipedia/commons/thumb/8/8b/Alcal%C3%A1_de_Henares_%28RPS_16-10-2011%29_Patrimonio_de_la_Humanidad.jpg/250px-Alcal%C3%A1_de_Henares_%28RPS_16-10-2011%29_Patrimonio_de_la_Humanidad.jpg
Unexpected status code: 403
https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/Alcal%C3%A1_de_Henares_%28RPS_11-02-2018%29_Comparsa_Cervantina_ante_el_Ayuntamiento.jpg/250px-Alcal%C3%A1_de_Henares_%28RPS_11-02-2018%29_Comparsa_Cervantina_ante_el_Ayuntamiento.jpg
Unexpected status code: 403
https://upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Alcal%C3%A1_de_Henares_%28RPS_22-10-2011%29_yacimiento_arqueol%C3%B3gico_Villa_Romana_del_Val.jpg/250px-Alcal%C3%A1_de_Henares_%28RPS_22-10-2011%29_yacimiento_arqueol%C3%B3gico_Villa_Romana_del_Val.jpg
Unexpected statu

# Library of Congress Collection

In [4]:
COLLECTION_URL = "https://www.loc.gov/collections/fsa-owi-color-photographs"
TEMP_FOLDER = os.path.join(os.getcwd(), "_TEMP")
OUTPUT_PATH = os.path.join(os.getcwd(), "fsa-owi-color-photographs.json")

print(f"Processing {COLLECTION_URL}")

# Get urls
def process_page(url, page_index):
    response = requests.get(f"{url}/?fa=access-restricted:false&&st=list&c=150&sp={page_index}")
    if response.status_code != 200:
        return None
    else:
        soup = BeautifulSoup(response.text, 'html.parser')
        element = soup.find(class_ = "search-results")
        if not element:
            return None
        else:
            images = element.find_all('img', class_ = "iconic")
            img_srcs = [img['src'] for img in images if 'src' in img.attrs]
            if not img_srcs:
                return None
            return img_srcs

image_list = []
found_last_page = False
count = 1
while found_last_page == False:
    rep = process_page(COLLECTION_URL, count)
    if rep == None:
        found_last_page = True
        break
    else:
        image_list.extend(rep)
        count = count + 1

print(f"Found {len(image_list)} images...")

if os.path.isdir(TEMP_FOLDER) == False:
    os.makedirs(TEMP_FOLDER)

output_data = {"images" : []}

for i, item in enumerate(image_list):
    image_url_thumb = item.split("#")[0]
    image_url = image_url_thumb.replace("_150px", "v")

    if image_url != "https://tile.loc.gov/storage-services/service/pnp/fsac/1a33000/1a33800/1a33881v.jpg":

        print_progress_bar(i, len(image_list) - 1, f"Treating {os.path.basename(image_url)}")

        to_add = {"url" : image_url}
        jhutils.online_files.download(image_url, dir = TEMP_FOLDER)
        img_info = get_image_info(os.path.join(TEMP_FOLDER, os.path.basename(image_url)))
        to_add["data"] = img_info

        jhutils.online_files.download(image_url_thumb, dir = TEMP_FOLDER)
        thumb_info = get_image_info(os.path.join(TEMP_FOLDER, os.path.basename(image_url_thumb)))
        to_add["thumbnail"] = {"url" : image_url_thumb, "data" : thumb_info}

        output_data["images"].append(to_add)
        write_json(OUTPUT_PATH, output_data)

#shutil.rmtree(TEMP_FOLDER)


Processing https://www.loc.gov/collections/fsa-owi-color-photographs
Found 1500 images...
Attempt 1 failed with error: HTTPSConnectionPool(host='tile.loc.gov', port=443): Read timed out. (read timeout=10)
Attempt 1 failed with error: HTTPSConnectionPool(host='tile.loc.gov', port=443): Read timed out. (read timeout=10)
Attempt 2 failed with error: HTTPSConnectionPool(host='tile.loc.gov', port=443): Read timed out. (read timeout=10)
Attempt 3 failed with error: HTTPSConnectionPool(host='tile.loc.gov', port=443): Read timed out. (read timeout=10)


ReadTimeout: HTTPSConnectionPool(host='tile.loc.gov', port=443): Read timed out. (read timeout=10)

# Google open images

In [7]:
import csv
import json
import random
import requests

TEMP_FOLDER = os.path.join(os.getcwd(), "_TEMP")
OUTPUT_JSON = os.path.join(os.getcwd(), "google-open-images-200.json")

# Configuration
NUM_IMAGES = 10
CSV_PATH = "/Users/jacob/Documents/Repos/arvest-data-in-context/ml-notebooks/_local/workflows/corpora/01-creating-an-image-corpus/train-images-boxable.csv"
IMAGE_BASE_URL = "https://storage.googleapis.com/openimages/2018_04/train/{}.jpg"
CSV_LOCAL = os.path.join(TEMP_FOLDER, "train-images-boxable.csv")

# Step 2: Read image IDs and pick random ones
def get_random_image_urls(csv_path, num_images):
    with open(csv_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header
        image_ids = [row[0] for row in reader]
    sampled_ids = random.sample(image_ids, num_images)
    return [IMAGE_BASE_URL.format(img_id) for img_id in sampled_ids]

# Step 3: Write to JSON
def write_json(image_urls, output_path):
    data = {"images": [{"url": url} for url in image_urls]}
    with open(output_path, 'w') as f:
        json.dump(data, f, indent=2)


urls = get_random_image_urls(CSV_PATH, NUM_IMAGES)
write_json(urls, OUTPUT_JSON)

print(f"Created JSON with {NUM_IMAGES} image URLs.")

Created JSON with 10 image URLs.
